In [ ]:
from elasticsearch import Elasticsearch

import time

import pandas as pd
import plotly.express as px

In [ ]:
# Elastic host
es = Elasticsearch(
    hosts=[
            "https://localhost:9200"
    ],
    basic_auth=("elastic", "NES9DZ-QwhanXAQf9caV"),
#     use_ssl=True,
#     verify_certs=False,
    ca_certs="./ca.crt"
)

In [ ]:
# NOTE: input is {search_key: value, search_key2: value2}
# query function
def query(search_dict):
    processed_search_list = []
    
    # loop door alle search elements heen
    for k, v in search_dict.items():
        processed_search_list.append({"match_phrase" : {k : v}})
        
    # stel de uitkomst samen
    result = es.search(
    index = "search",
    size = 10000, # TODO: Zorg dat er een groter limit is dan 10000
    query = {
        "bool" : {
            "must": processed_search_list,},})
    
    return result

In [ ]:
# query testruimte
start_time = time.time()

# de query zelf
# test_result = query({"content_simplified":"ete", 
#                      "person_simplified":"FreDeriC BARBIER", 
#                      "country":"FR"})

test_result = query({"content_simplified":"groen",
                     "party":"SP",
                     "year":"2018"})

print("Got %d Hits:" % test_result['hits']['total']['value'])

for hit in test_result['hits']['hits']:
    print("%(person)s (%(party)s) \n(%(year)s-%(month)s-%(day)s) %(segment)s:\n %(content)s \n" % hit["_source"])

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# neemt een query als input en geeft een df terug voor plotly express
# NOTE: x en y zijn nog niet verwerkt
def get_ngram_df(query, x="year", y="scaled"):
    dates_dict = {}
    
    # loop door elk resultaat
    for hit in query:
        src = hit["_source"]
        
        # stel de date op (rond af op MAANDEN)
        # NOTE: 1 veranderen naar src["day"] voor specifieker
        date = "{}-{}-{}".format(src["year"], src["month"], 1)
        
        # tel hoeveel hits er zijn per date
        if date in dates_dict:
            
            dates_dict[date] += 1
            
        else:
            
            dates_dict[date] = 1
    
    # stel de dataframe op voor plotly express
    df = pd.DataFrame.from_dict((dates_dict), columns=['count'], orient='index').reset_index().rename(columns={'index': 'date'})
        
    return df.sort_values(by=['date'])

In [ ]:
# ngram testruimte
word_1 = "democracy"
word_2 = "democratie"

test_result_1 = query({"content_simplified":word_1})
test_result_2 = query({"content_simplified":word_2})

print("Got %d Hits:" % test_result_1['hits']['total']['value'])
print("Got %d Hits:" % test_result_2['hits']['total']['value'])

test_ngram_df_1 = get_ngram_df(test_result_1['hits']['hits'])
test_ngram_df_1["query"] = word_1
test_ngram_df_2 = get_ngram_df(test_result_2['hits']['hits'])
test_ngram_df_2["query"] = word_2

test_ngram_df = test_ngram_df_1.merge(test_ngram_df_2, "outer")

fig = px.line(test_ngram_df, x="date", y="count", color='query', title='Ngram (soort van)')

fig.show()